In [1]:
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import src.data.breathe_data as bd
import src.data.helpers as dh
import src.inference.helpers as ih
import src.modelling_ar.ar as ar
import src.inf_cutset_conditioning.helpers as cutseth
import src.inf_cutset_conditioning.cutset_cond_algs_learn_ar_change as cca_ar_change
import src.models.helpers as mh

In [31]:
# Load data
df = bd.load_meas_from_excel("BR_O2_FEV1_FEF2575_conservative_smoothing_with_idx")

INFO:root:* Checking for same day measurements *


In [3]:
id = "134"
dftmp, start_idx, end_idx = dh.find_longest_consec_series(df[df.ID == id], n_days=3)


ecfev1_noise_model_suffix = "_std_add_mult_fev1"
ar_prior = "breathe (2 days model, ecFEV1 addmultnoise, ecFEF25-75)"
ar_change_cpt_suffix = "_shape_factor1"

(
    # p_M_given_D,
    # log_p_D_given_M,
    # AR_given_M_and_D,
    # AR_given_M_and_all_D,
    log_p_S_given_D,
    res_dict,
) = cca_ar_change.run_long_noise_model_through_time(
    dftmp,
    ar_prior=ar_prior,
    ar_change_cpt_suffix=ar_change_cpt_suffix,
    ecfev1_noise_model_suffix=ecfev1_noise_model_suffix,
)

Warning - min_possible_hfev1_under_model: 6
134 - Time for 18 entries: 728.41 s


In [ ]:
# Read p_s_given_d.json
import json

path = f"{dh.get_path_to_src()}inf_cutset_conditioning/p_s_given_d.json"
# Read text from file
# The file is a dump of dictionaries written like {'key': array([1,2,3])}
# Convert this to a dict of numpy arrays

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [47]:
import json
import numpy as np

# Load the JSON file
with open(path, "r") as file:
    data = json.load(file)

# Convert lists to NumPy arrays
for key in data:
    data[key] = np.array(data[key])

# Replace 'nan' with np.nan and "-inf" with -np.inf
for key in data:
    data[key][np.where(data[key] == "nan")] = np.nan
    data[key][np.where(data[key] == "-inf")] = -np.inf

# REmove dtypes from data
data = {key: data[key].astype(float) for key in data}

# Remove entries with only nan values
data = {key: data[key] for key in data if not np.all(np.isnan(data[key]))}

In [49]:
# Find entries that are in df.ID.unique() but not in data.keys()
missing_ids = np.setdiff1d(df.ID.unique(), list(data.keys()))
missing_ids

array(['113', '115', '123', '125', '126', '130', '134', '141', '144',
       '151', '156', '166', '168', '170', '177', '178', '182', '185',
       '189', '190', '196', '199', '202', '209', '212', '213', '221',
       '230', '233', '237', '240', '242', '244', '245', '250', '251',
       '254', '255', '257', '260', '262', '266', '267', '272', '273',
       '274', '275', '276', '279', '280', '281', '288', '290', '291',
       '292', '296', '297', '299', '300', '302', '303', '312', '313',
       '316', '320', '333', '337', '338', '340', '351', '353', '355',
       '358', '359', '370', '372', '381', '382', '386', '392', '401',
       '402', '405', '406', '407', '420', '425', '426', '428', '429',
       '443', '444', '448', '454', '455', '461', '467', '469', '480',
       '481', '482', '483', '485', '489', '490', '492', '499', '502',
       '505', '506', '507', '508', '509', '510', '512', '513', '514',
       '515', '516', '517', '518', '520', '523', '525', '527', '529',
       '530', '531',